In [2]:
import numpy as np
from sklearn.datasets import load_digits
import theano.tensor as T
import theano

In [3]:
data=load_digits()

In [6]:
data.keys()

['images', 'data', 'target_names', 'DESCR', 'target']

In [9]:
x=data['data']
y=data['target']

In [13]:
def recode_y(y):
    final=np.zeros((y.shape[0],10))
    for i in range(10):
        final[:,i]=(y==i)+0
    return final
recode_y(y)

array([[ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  1.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  1., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  1.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.],
       [ 0.,  0.,  0., ...,  0.,  1.,  0.]])

In [15]:
from sklearn.cross_validation import train_test_split
np.random.seed(123)
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.20)
x_train,x_val,y_train,y_val=train_test_split(x_train,y_train,test_size=0.25)

In [35]:
y_train_ohe=recode_y(y_train)

In [59]:
#creating a container for the data storage variable

def floatX(x):
    ''' theano is very particular about the data type of the shared variable espescially the data type of the 
    shared variable is supposed to be theano.config.floatX
    
    this function converts the input array in a theano compatiable array
    '''
    return np.asarray(x,dtype=theano.config.floatX)

input_dim=64
hidden_dim=625
output_dim=10

learning_rate=0.01

X=T.fmatrix()
Y=T.fmatrix()

#creating the initial weight variables

# W1=theano.shared(np.random.rand(input_dim,hidden_dim))
# b1=floatX(theano.shared(np.zeros(hidden_dim)))

# W2=floatX(theano.shared(np.random.rand(hidden_dim,output_dim)))
# b2=floatX(theano.shared(np.zeros(output_dim)))



def floatX(x):
    return np.asarray(x,dtype=theano.config.floatX)

def init_weights(shape):
    return theano.shared(floatX(np.random.rand(*shape)))

def init_bias(shape):
    return theano.shared(np.asarray(np.zeros(shape),dtype=theano.config.floatX))


W1=init_weights((64,100))
b1=init_bias(100)

W2=init_weights((100,10))
b2=init_bias(10)





def model(X,W1,b1,W2,b2):
    a1=T.dot(X,W1)+b1
    z1=T.nnet.sigmoid(a1)
    a2=T.dot(z1,W2)+b2
    z2=T.nnet.softmax(a2)
    return z2


py_x=model(X,W1,b1,W2,b2)

nn_pred=T.argmax(py_x,axis=1)

cost=T.mean(T.nnet.categorical_crossentropy(py_x,Y))

dW1=T.grad(cost,W1)
db1=T.grad(cost,b1)

dW2=T.grad(cost,W2)
db2=T.grad(cost,b2)

updates=[[W1,W1-learning_rate*dW1],[b1,b1-learning_rate*db1],[W2,W2-learning_rate*dW2],[b2,b2-learning_rate*db2]]

# updates=[[W1,W1-learning_rate*dW1],[W2,W2-learning_rate*dW2]]


train=theano.function(inputs=[X,Y],outputs=[cost],updates=updates,allow_input_downcast=True)

prediction=theano.function(inputs=[X],outputs=[nn_pred],allow_input_downcast=True)


for i in range(100):
    cost=train(x_train,y_train_ohe)
    print cost
    predicted_output=prediction(x_val)
    predicted_output





[array(5.2208969482734435)]
[array(4.8231805133849)]
[array(4.509148421985462)]
[array(4.248155856310939)]
[array(4.028156193641526)]
[array(3.8417159361910977)]
[array(3.6820139044344233)]
[array(3.5431104515184186)]
[array(3.420501849533105)]
[array(3.311087294916551)]
[array(3.212806315798694)]
[array(3.1242516710688486)]
[array(3.044383140207211)]
[array(2.972353726081241)]
[array(2.907420089386911)]
[array(2.848904625353148)]
[array(2.7961838734302082)]
[array(2.748687096346099)]
[array(2.7058964256973637)]
[array(2.667345126137365)]
[array(2.6326135074263375)]
[array(2.6013233709045425)]
[array(2.573132189503291)]
[array(2.547727971428409)]
[array(2.524825306465852)]
[array(2.50416266942517)]
[array(2.485500767742389)]
[array(2.4686215899042128)]
[array(2.4533278056838963)]
[array(2.439442237189989)]
[array(2.4268072148728517)]
[array(2.41528372330446)]
[array(2.4047503120507687)]
[array(2.3951017935593564)]
[array(2.386247775714193)]
[array(2.378111087135425)]
[array(2.370626153

In [46]:
trX=x_train
trY=y_train_ohe

In [65]:
import theano
from theano import tensor as T
import numpy as np


input_dim=64
hidden_dim=625
output_dim=10



def floatX(X):
    return np.asarray(X, dtype=theano.config.floatX)

def init_weights(shape):
    return theano.shared(floatX(np.random.randn(*shape) * 0.01))

def sgd(cost, params, lr=0.05):
    grads = T.grad(cost=cost, wrt=params)
    updates = []
    for p, g in zip(params, grads):
        updates.append([p, p - g * lr])
    return updates


def relu(x):
    return theano.tensor.switch(x<0, 0, x)

def model(X, w_h, w_o):
#     h = T.nnet.sigmoid(T.dot(X, w_h))
    h = relu(T.dot(X, w_h))
    pyx = T.nnet.softmax(T.dot(h, w_o))
    return pyx


X = T.fmatrix()
Y = T.fmatrix()

w_h = init_weights((input_dim, hidden_dim))
w_o = init_weights((hidden_dim, output_dim))

py_x = model(X, w_h, w_o)
y_x = T.argmax(py_x, axis=1)

cost = T.mean(T.nnet.categorical_crossentropy(py_x, Y))
params = [w_h, w_o]
updates = sgd(cost, params)

train = theano.function(inputs=[X, Y], outputs=cost, updates=updates, allow_input_downcast=True)
predict = theano.function(inputs=[X], outputs=y_x, allow_input_downcast=True)


best_param={}
best_accuracy=0



for i in range(200):
    for start, end in zip(range(0, len(trX), 128), range(128, len(trX), 128)):
        cost = train(trX[start:end], trY[start:end])

    #print np.mean(np.argmax(teY, axis=1) == predict(teX))
#     print cost
#     print predict(x_val)
    accuracy=np.mean(predict(x_val)==y_val)
    if best_accuracy<accuracy:
        best_accuracy=accuracy
        best_param["W1"]=W1.get_value()
        best_param["W2"]=W2.get_value()




In [66]:
best_accuracy

0.96666666666666667

So the best accuracy on training this network,I got is of 96%. Now furhter tuning in terms of learning rate and the number of iteration can be done. But i am not doing it right now.

For the same network archetecture but now using a reLu unit as the hidden activation function i get 96.7% accuracy on the test set.